In [3]:

!pip show opencv-python
!which python



Name: opencv-python
Version: 4.10.0.84
Summary: Wrapper package for OpenCV python bindings.
Home-page: https://github.com/opencv/opencv-python
Author: 
Author-email: 
License: Apache 2.0
Location: /Users/pirozhok/Desktop/study/Fingers_Detector/jupyter_env/lib/python3.12/site-packages
Requires: numpy
Required-by: 
/Users/pirozhok/Desktop/study/Fingers_Detector/jupyter_env/bin/python


In [11]:
!{sys.executable} -m pip install opencv-python


  Using cached opencv-python-4.10.0.84.tar.gz (95.1 MB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... error
  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> [33 lines of output]
      Traceback (most recent call last):
        File "/opt/anaconda3/lib/python3.12/site-packages/pip/_vendor/pyproject_hooks/_in_process/_in_process.py", line 353, in <module>
          main()
        File "/opt/anaconda3/lib/python3.12/site-packages/pip/_vendor/pyproject_hooks/_in_process/_in_process.py", line 335, in main
          json_out['return_val'] = hook(**hook_input['kwargs'])
                                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
        File "/opt/anaconda3/lib/python3.12/site-packages/pip/_vendor/pyproject_hooks/_in_process/_in_process.py", line 112, in get_requires_for_build_wheel
          backend = _build_backend()
                    ^^^^^^^^^^^^^^^^
        File "/op

In [1]:
import cv2


In [3]:
import numpy as np


In [ ]:
from sklearn.metrics import pairwise



In [ ]:
background = None

accumulated_weight = 0.5

roi_top = 20
roi_bottom = 300
roi_right = 300
roi_left = 600

In [ ]:
def calc_acuum_avg(frame,accumulated_weight):
    global background

    if background is None:
        background = frame.copy().astype('float')
        return None

    cv2.accumulateWeighted(frame,background,accumulated_weight)

In [11]:
def segment(frame, threshold=25):
    diff = cv2.absdiff(background.astype('uint8'), frame)

    ret,tresholded = cv2.threshold(diff, threshold_min, 255, cv2.THRESH_BINARY)

    image,contours,hierarchy = cv2.findContours(thresholded.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    if len(contours) == 0:
        return None

    else:
        # Assuming the largest external contour in roi is th hand
        hand_segment = max(contours,key=cv2.contourArea)

        return (trheshold, hand_segment)